# Scraping French Fake News

This notebook use scrapy classes to retrieve latest news content from :
- Le Gorafi (societe, politique)
- Nord Presse.be (France)
- BuzzBeed

2 possible sources : 
- pages links and next page 
- RSS feed

## Helper functions

### clean_file : Save older scaping result

In [1]:
# save before scraping

import shutil
# importing os module
import os
# import datetime module
import datetime


def clean_file(path_file_name):
    '''
    Clean file already traited : rename file with date
    '''
    try:
        d = datetime.datetime.now()
        str_date = '_' + d.strftime("%Y%m%d_%H_%M_%S")
        res_re = re.search('\.\w+$',path_file_name)
        path_file_name_saved = \
            path_file_name[0:res_re.start()] + str_date + res_re.group(0)
            
        shutil.move(path_file_name, path_file_name_saved) 
        print('File {} moved!'.format(path_file_name_saved))
    except:
        print('File {} does not exist!'.format(path_file_name))
        


### run_spider : running spider several times 

In [2]:
import scrapy
import scrapy.crawler as crawler
from multiprocessing import Process, Queue
from twisted.internet import reactor

# the wrapper to make it run more times
def run_spider(spider):
    '''
    function to run several times scraping process
    '''
    def f(q):
        try:
            runner = crawler.CrawlerRunner()
            deferred = runner.crawl(spider)
            deferred.addBoth(lambda _: reactor.stop())
            reactor.run()
            q.put(None)
        except Exception as e:
            q.put(e)

    q = Queue()
    p = Process(target=f, args=(q,))
    p.start()
    result = q.get()
    p.join()

    if result is not None:
        raise result

## Scraping Legorafi

### Class definitions

In [3]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags

class GorafiItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output)
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=Join()
    )

In [4]:
class GorafiSpider(scrapy.Spider):
    '''
    Spider to scrap over Le Gorafi webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_gorafi_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_gorafi.json'
  }
    # urls to scrap
    start_urls = [
        'http://www.legorafi.fr/2019/12/17/psycho-comment-guerir-dun-chagrin-damour-en-donnant-tout-son-argent-au-gorafi/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=GorafiItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "LeGorafi")
        
        l.add_xpath('author', 
            "//section[@class='metas container']/span[@class='context']/a")
        
        l.add_xpath("title", "//h1")
        
        l.add_xpath("theme", "//a[@rel='category tag']")
        
        l.add_xpath("description", ".//div[@class='intro']/p")
        
        l.add_xpath("date_published", 
            "//section[@class='metas container']/span[@class='context']",
            re="[0-9]+/[0-9]+/[0-9]+")
       
        l.add_xpath("body", "//div[@class='content']/p")
        
        yield l.load_item()

In [5]:
import scrapy

class GorafiRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from Le Gorafi RSS
    '''
    name = "gorafi_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'gorafi_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'http://www.legorafi.fr/feed/',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [6]:
import scrapy
import re

class GorafiPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Le Gorafi pages from selected category
    Configure : 
    - GorafiPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "gorafi_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_gorafi.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'http://www.legorafi.fr/category/france/societe/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        for post in response.xpath('//article/h2'):
            yield {
                'link': post.xpath('a/@href').extract_first()
            }
            
        next_page = response.xpath(
            '//a[@class="next page-numbers"]/@href').get()
        
        if next_page is not None:
            try:
                num_next_page = int(re.search("(?<=/)\d+(?=/$)", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [8]:
# path to results file for urls
PATH_RSS_PAGES_GORAFI = '../../data/gorafi_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_GORAFI = '../../data/gorafi_rss_pages.json'

#### Scraping urls links

In [9]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_GORAFI)

File ../../data/gorafi_rss_urls.json does not exist!


In [10]:
# Configure spider
GorafiRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_GORAFI
}
# scraping
run_spider(GorafiRssSpider)

In [11]:
import pandas as pd

df_rss_gorafi = pd.read_json(PATH_RSS_PAGES_GORAFI)
df_rss_gorafi

,link,pubDate,title
0,http://www.legorafi.fr/2019/12/20/le-pere-noel...,"Fri, 20 Dec 2019 13:00:49 +0000",Le Père Noël découvre stupéfait l’existence de...
1,http://www.legorafi.fr/2019/12/20/gorafi-magaz...,"Fri, 20 Dec 2019 09:00:04 +0000",Gorafi Magazine : Partir à la retraite à 115 ans
2,http://www.legorafi.fr/2019/12/19/noel-pour-co...,"Thu, 19 Dec 2019 13:00:32 +0000","Noël – Pour coller à la réalité, les magasins ..."
3,http://www.legorafi.fr/2019/12/19/le-gouvernem...,"Thu, 19 Dec 2019 09:22:54 +0000",Le gouvernement autorise les chauffeurs des bu...
4,http://www.legorafi.fr/2019/12/18/espagne-18-m...,"Wed, 18 Dec 2019 09:14:41 +0000",Espagne – 18 morts dans la traditionnelle bata...
5,http://www.legorafi.fr/2019/12/17/psycho-comme...,"Tue, 17 Dec 2019 09:03:56 +0000",Psycho : Comment guérir d’un chagrin d’amour e...
6,http://www.legorafi.fr/2019/12/16/apres-sa-dem...,"Mon, 16 Dec 2019 13:00:39 +0000",Après sa démission Jean-Paul Delevoye savoure ...
7,http://www.legorafi.fr/2019/12/16/le-tfc-va-sa...,"Mon, 16 Dec 2019 09:04:56 +0000",Le TFC va sacrifier un de ses joueurs pour cal...
8,http://www.legorafi.fr/2019/12/16/horoscope-du...,"Mon, 16 Dec 2019 08:31:26 +0000",Horoscope du 16 décembre 2019
9,http://www.legorafi.fr/2019/12/13/les-climato-...,"Fri, 13 Dec 2019 14:30:12 +0000",Les climato-sceptiques présentent leur Greta ...


In [12]:
df_rss_gorafi.shape

(20, 3)

#### Scraping RSS news

In [ ]:
# clean (move file if exist)
clean_file(PATH_RSS_NEWS_GORAFI)

In [13]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_rss_gorafi["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_GORAFI
}

# scraping
run_spider(GorafiSpider)

In [14]:
import pandas as pd

df_gorafi = pd.read_json(PATH_RSS_NEWS_GORAFI)
df_gorafi.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« Je n’en revenais pas… C’est un lutin stagiai...,20/12/2019,A quatre jours de la distribution des cadeaux ...,LeGorafi,Société,Le Père Noël découvre stupéfait l’existence de...,http://www.legorafi.fr/2019/12/20/le-pere-noel...
1,La Rédaction,NaN,20/12/2019,NaN,LeGorafi,Magazine,Gorafi Magazine : Partir à la retraite à 115 ans,http://www.legorafi.fr/2019/12/20/gorafi-magaz...
2,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
3,La Rédaction,Ainsi les chauffeurs sont autorisés à prendre ...,19/12/2019,Paris – Pour aider les Français à pouvoir prof...,LeGorafi,Société,Le gouvernement autorise les chauffeurs des bu...,http://www.legorafi.fr/2019/12/19/le-gouvernem...
4,La Rédaction,Oliver Sadran espère ainsi conjurer le mauvais...,16/12/2019,Aux grands maux les grands remèdes. Afin de ca...,LeGorafi,Sports,Le TFC va sacrifier un de ses joueurs pour cal...,http://www.legorafi.fr/2019/12/16/le-tfc-va-sa...


In [15]:
df_gorafi.shape

(20, 8)

### Scraping urls into webpages 

#### Definitions

In [190]:
URL_PAGES_GORAFI_SOCIETE = 'http://www.legorafi.fr/category/france/societe/'
# declare your data location
PATH_PAGES_GORAFI_SOCIETE = '../../data/pages_gorafi.json'
PATH_NEWS_GORAFI_SOCIETE = '../../data/gorafi_societe.json'

URL_PAGES_GORAFI_POLITIQUE = 'http://www.legorafi.fr/category/france/politique/'
# declare your data location
PATH_PAGES_GORAFI_POLITIQUE = '../../data/pages_gorafi_politique.json'
PATH_NEWS_GORAFI_POLITIQUE = '../../data/gorafi_politique.json'

#### Le Gorafi Société

##### Scraping links

In [153]:
# clean (move file if exist)
clean_file(PATH_PAGES_GORAFI_SOCIETE)

File ../../data/pages_gorafi_20191225_10_47_30.json moved!


In [104]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_SOCIETE
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_SOCIETE
    }
GorafiPageSpider.num_max_pages = 50

# clean last output
clean_file(PATH_PAGES_GORAFI_SOCIETE)
# scraping page urls LeGorafi
run_spider(GorafiPageSpider)

In [161]:
import pandas as pd

df_gorafi_pages = pd.read_json(PATH_PAGES_GORAFI_SOCIETE)
df_gorafi_pages.head()

,link
0,http://www.legorafi.fr/2019/12/19/noel-pour-co...
1,http://www.legorafi.fr/2019/12/19/le-gouvernem...
2,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,http://www.legorafi.fr/2019/12/12/test-quel-es...
4,http://www.legorafi.fr/2019/12/11/plusieurs-bu...


In [166]:
df_gorafi_pages.shape

(246, 1)

##### Scraping news

In [ ]:
# clean last output
clean_file(PATH_NEWS_GORAFI_SOCIETE)

In [167]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_SOCIETE
    }
# scraping
run_spider(GorafiSpider)

In [195]:
import pandas as pd

df_gorafi_soc = pd.read_json(PATH_NEWS_GORAFI_SOCIETE)
df_gorafi_soc.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
1,La Rédaction,"Oui, car Mathias a économisé pas moins de 5 eu...",10/12/2019,"Alors qu’il rentrait d’un week-end, Mathias a ...",LeGorafi,Société,Il économise 5 euros en achetant au duty-free ...,http://www.legorafi.fr/2019/12/10/il-economise...
2,La Rédaction,1/ Faire un beau chèque en s’appliquant pour l...,17/12/2019,L’échec amoureux n’a plus de secret pour vous ...,LeGorafi,Société,Psycho : Comment guérir d’un chagrin d’amour e...,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,La Rédaction,\nwindow.beOpAsyncInit = function() {\n BeOpS...,12/12/2019,"Depuis les annonces d’Edouard Philippe, vous v...",LeGorafi,Société,Test : quel est l’âge idéal pour mourir avant ...,http://www.legorafi.fr/2019/12/12/test-quel-es...
4,La Rédaction,"Selon les services sanitaires, se sont plusieu...",11/12/2019,Paris – La chaîne de restauration rapide lancé...,LeGorafi,Société,Plusieurs Burger Quiz fermés après une visite ...,http://www.legorafi.fr/2019/12/11/plusieurs-bu...


In [196]:
df_gorafi_soc.shape

(246, 8)

In [197]:
df_gorafi_soc[df_gorafi_soc["body"].isna()].shape[0]

0

#### Le Gorafi Politique

##### Scraping links

In [ ]:
# clean last output
clean_file(PATH_PAGES_GORAFI_POLITIQUE)

In [155]:
# configure scraping
GorafiPageSpider.url_first_page = URL_PAGES_GORAFI_POLITIQUE
GorafiPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_GORAFI_POLITIQUE
    }
GorafiPageSpider.num_max_pages = 50

# move file if exist
clean_file(PATH_PAGES_GORAFI_POLITIQUE)

# scraping pages LeGorafi
run_spider(GorafiPageSpider)

File ../../data/pages_gorafi_politique.json does not exist!


In [163]:
import pandas as pd

df_gorafi_pages_pol = pd.read_json(PATH_PAGES_GORAFI_POLITIQUE)
df_gorafi_pages_pol.head()

,link
0,http://www.legorafi.fr/2019/12/11/jean-paul-de...
1,http://www.legorafi.fr/2019/12/09/jean-paul-de...
2,http://www.legorafi.fr/2019/12/05/christophe-c...
3,http://www.legorafi.fr/2019/11/25/lrem-edouard...
4,http://www.legorafi.fr/2019/11/22/inquiete-par...


In [170]:
df_gorafi_pages_pol.shape

(246, 1)

##### Scraping news

In [ ]:
# clean last output
clean_file(PATH_NEWS_GORAFI_POLITIQUE)

In [171]:
# configure : add all retrieved links to Spider
GorafiSpider.start_urls = df_gorafi_pages_pol["link"].tolist()
GorafiSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_GORAFI_POLITIQUE
    }
# scraping
run_spider(GorafiSpider)

In [191]:
import pandas as pd

df_gorafi_pol = pd.read_json(PATH_NEWS_GORAFI_POLITIQUE)
df_gorafi_pol.head()

,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« C’est juste un petit cadeau pour le service ...,09/12/2019,Paris – Nouvelle polémique pour Jean-Paul Dele...,LeGorafi,Politique,Jean-Paul Delevoye a oublié de préciser qu’il ...,http://www.legorafi.fr/2019/12/09/jean-paul-de...
1,La Rédaction,De nombreuses sources citent Marlène Schiappa ...,25/11/2019,"De nombreux détails croustillants, concernant ...",LeGorafi,Politique,LREM : Édouard Philippe rebaptisé Eduardo Fili...,http://www.legorafi.fr/2019/11/25/lrem-edouard...
2,La Rédaction,« Oui c’est un peu regrettable mais j’ai quitt...,11/12/2019,"Paris – Face aux polémiques naissantes, Jean-P...",LeGorafi,Politique,Jean-Paul Delevoye contraint de démissionner d...,http://www.legorafi.fr/2019/12/11/jean-paul-de...
3,La Rédaction,« La précarité qu’ils traversent leur sera de ...,13/11/2019,Alors que la mobilisation étudiante prend de l...,LeGorafi,Politique,Emmanuel Macron aux étudiants « Je n’adore pas...,http://www.legorafi.fr/2019/11/13/emmanuel-mac...
4,La Rédaction,Alors que de nombreux manifestants sont attend...,05/12/2019,Après que l’Etat a commandé de nouveaux LBD à ...,LeGorafi,Politique,Christophe Castaner promet que les LBD « feron...,http://www.legorafi.fr/2019/12/05/christophe-c...


In [192]:
df_gorafi_pol.shape

(246, 8)

In [193]:
df_gorafi_pol[df_gorafi_pol["body"].isna()].shape[0]

0

## Scraping Nordpresse.be

### Class definitions

In [106]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags


class NordpresseItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output) for NordPresse.be
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=lambda v: v[-1]
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(output_processor=TakeFirst())
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags, 
                                   lambda v: v.replace(u'\xa0', u' '),
                                  lambda v: v.strip()),
        output_processor=Join()
    )

In [107]:
class NordpresseSpider(scrapy.Spider):
    '''
    Spider to scrap over Nordpresse webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_nordpresse_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_nordpresse.json'
    }
    
    # urls to scrap
    start_urls = [
        'https://nordpresse.be/category/france/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=NordpresseItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "Nordpresse")
        
        #<div class="td-post-author-name td-post-author-no-dot">
        #  <div class="td-author-by">By</div> 
        #  <a href="https://nordpresse.be/...">Stephane VERON</a></div>
        l.add_xpath('author', 
            '//div[@class="td-post-author-name td-post-author-no-dot"]/a')
        
        l.add_xpath("title", "//h1")
        
        #<div class="td-post-header">
        #    <ul class="td-category">
        #        <li class="entry-category"><a href="https://nordpresse.be/category/france/">France</a></li>
        #        <li class="entry-category"><a href="https://nordpresse.be/category/politique/">Politique</a></li></ul>
            
        l.add_xpath("theme", '//div[@class="td-post-header"]/ul/li/a')
        
        #l.add_xpath("description", ".//div[@class='intro']/p")
        l.add_value('description', "")
        
        #<meta property="article:published_time" content="2019-12-15T14:21:58+00:00">
        l.add_xpath("date_published", 
            '//meta[@property="article:published_time"]/@content',
            re="[0-9]+-[0-9]+-[0-9]+")
       
        l.add_xpath("body", "//div[@class='td-post-content']/p")
        
        yield l.load_item()

In [108]:
import scrapy

class NordpresseRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from Nordpresse RSS
    '''
    name = "nordpresse_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'nordpresse_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'https://nordpresse.be/feed',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [109]:
import scrapy
import re

class NordpressePageSpider(scrapy.Spider):
    '''
    Spider to scrap all Nordpresse pages from selected category
    Configure : 
    - NordpressePageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "nordpresse_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_nordpresse.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'https://nordpresse.be/category/france/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        
        # //div[@class="td-pb-span8 td-main-content"]//h3[@class="entry-title td-module-title"]/a[@rel="bookmark"]
        for post in response.xpath('//div[@class="td-pb-span8 td-main-content"]//h3[@class="entry-title td-module-title"]'):
            yield {
                'link': post.xpath('a[@rel="bookmark"]/@href').extract_first()
            }
        # <a href="https://nordpresse.be/category/france/page/3/"><i class="td-icon-menu-right"></i></a>
        next_page = response.xpath(
            '//i[@class="td-icon-menu-right"]//parent::a/@href').get()
        if next_page is not None:
            try:
                num_next_page = int(re.search("(?<=/)\d+(?=/$)", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [20]:
# path to results file for urls
PATH_RSS_PAGES_NORDPRESSE = '../../data/nordpresse_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_NORDPRESSE = '../../data/nordpresse_rss_pages.json'

#### Scraping RSS urls links

In [21]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_NORDPRESSE)

File ../../data/nordpresse_rss_urls.json does not exist!


In [22]:
# Configure spider
NordpresseRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_NORDPRESSE
}
# scraping
run_spider(NordpresseRssSpider)

In [23]:
import pandas as pd

df_rss_nordpresse = pd.read_json(PATH_RSS_PAGES_NORDPRESSE)
df_rss_nordpresse

,link,pubDate,title
0,https://nordpresse.be/lrem-accuse-de-faire-de-...,"Wed, 25 Dec 2019 20:32:10 +0000",LREM accusé de faire de l’ombre à la celèbre é...
1,https://nordpresse.be/segolene-royale-desormai...,"Wed, 25 Dec 2019 19:00:57 +0000",Segolène Royale désormais pourvue d’un antivol
2,https://nordpresse.be/emmanuel-macron-renonce-...,"Wed, 25 Dec 2019 09:47:28 +0000",Emmanuel Macron renonce à sa retraite: « Je va...
3,https://nordpresse.be/ce-cosplay-est-plutot-pa...,"Wed, 25 Dec 2019 08:53:17 +0000",Ce cosplay est plutôt pas mal non ?
4,https://nordpresse.be/melenchon-lannonce-mouve...,"Wed, 25 Dec 2019 07:37:16 +0000",Mélenchon l’annonce: Mouvement de grève généra...
5,https://nordpresse.be/decouvrez-le-sapin-de-no...,"Tue, 24 Dec 2019 23:07:32 +0000",Découvrez le sapin de Noël ecoresponsable de G...
6,https://nordpresse.be/macron-aurait-fait-bloqu...,"Tue, 24 Dec 2019 10:36:46 +0000",Macron aurait fait bloquer Mêmes Décentralisés...
7,https://nordpresse.be/un-belge-invente-le-supp...,"Sun, 22 Dec 2019 22:22:09 +0000",Un Belge invente le suppositoire effervescent ...
8,https://nordpresse.be/tibo-in-shape/,"Sun, 22 Dec 2019 20:59:58 +0000",Tibo in Shape
9,https://nordpresse.be/macron-compte-rester-au-...,"Sun, 22 Dec 2019 01:01:49 +0000",Macron compte rester au pouvoir jusque 64 ans ...


In [56]:
df_rss_nordpresse.shape

(10, 3)

#### Scraping RSS news

In [80]:
# clean (move file if exist)
clean_file(PATH_RSS_NEWS_NORDPRESSE)

File ../../data/nordpresse_rss_pages_20191226_21_54_31.json moved!


In [81]:
# configure : add all retrieved links to Spider
NordpresseSpider.start_urls = df_rss_nordpresse["link"].tolist()
NordpresseSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_NORDPRESSE
}

# scraping
run_spider(NordpresseSpider)

In [82]:
import pandas as pd

df_nordpresse = pd.read_json(PATH_RSS_NEWS_NORDPRESSE)
df_nordpresse.head()

,author,body,date_published,source,theme,title,url
0,Kael A,"En effet, l’équipe de production s’inquiète d’...",2019-12-25,Nordpresse,Non classé,LREM accusé de faire de l’ombre à la celèbre é...,https://nordpresse.be/lrem-accuse-de-faire-de-...
1,Vincent Flibustier,Il s’appelle Mathieu Alias le testeur temporel...,2019-12-25,Nordpresse,Complot,Ce cosplay est plutôt pas mal non ?,https://nordpresse.be/ce-cosplay-est-plutot-pa...
2,Sw,Personne ne sait pourquoi elle est en train de...,2019-12-25,Nordpresse,Non classé,Segolène Royale désormais pourvue d’un antivol,https://nordpresse.be/segolene-royale-desormai...
3,ToLa,Emmanuel Macron a récemment déclaré avoir reno...,2019-12-25,Nordpresse,Gilets Jaunes,Emmanuel Macron renonce à sa retraite: « Je va...,https://nordpresse.be/emmanuel-macron-renonce-...
4,Vincent Flibustier,NaN,2019-12-24,Nordpresse,Non classé,Découvrez le sapin de Noël ecoresponsable de G...,https://nordpresse.be/decouvrez-le-sapin-de-no...


In [84]:
df_nordpresse.at[0,"body"]

'En effet, l’équipe de production s’inquiète d’une évidente concurrence déloyale et ne compte pas en rester là à l’attention de la formation LREM, lorsque notre reporter s’est présenté pour recevoir les doléances, il a été accueilli par cette phrase de l’un des membre du personnel : » On ne sait pas ce qu’ils mijotent, mais ça sent pas très bon » Suivi par la cinglante déclaration  » Dans les médias, on ne parle que de ça, LREM et son ensemble inépuisable de casseroles, cela ne peux plus durer, ils mettent en danger les vrais professionnels  » *Certaines déclaration n’on pas été retransmises par notre reporter .  '

In [47]:
df_nordpresse.at[0,"url"]

'https://nordpresse.be/lrem-accuse-de-faire-de-lombre-a-la-celebre-emission-cauchemar-en-cuisine-a-cause-du-nombre-de-ses-casseroles/'

In [40]:
df_nordpresse.shape

(10, 7)

### Scraping urls into webpages 

#### Definitions

In [110]:
URL_PAGES_NORDPRESSE = 'https://nordpresse.be/category/france/'
# declare your data location
PATH_PAGES_NORDPRESSE = '../../data/pages_nordpresse.json'
PATH_NEWS_NORDPRESSE = '../../data/nordpresse.json'

#### Scraping url links

In [117]:
# clean (move file if exist)
clean_file(PATH_PAGES_NORDPRESSE)

File ../../data/pages_nordpresse_20191227_16_17_23.json moved!


In [ ]:
# configure scraping
NordpressePageSpider.url_first_page = URL_PAGES_NORDPRESSE
NordpressePageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_NORDPRESSE
    }
NordpressePageSpider.num_max_pages = 50

# clean last output
clean_file(PATH_PAGES_NORDPRESSE)
# scraping page urls
run_spider(NordpressePageSpider)

In [119]:
import pandas as pd

df_nordpresse_pages = pd.read_json(PATH_PAGES_NORDPRESSE)
df_nordpresse_pages.head()

,link
0,https://nordpresse.be/emmanuel-macron-renonce-...
1,https://nordpresse.be/j-p-delevoye-toujours-au...
2,https://nordpresse.be/retraites-decourages-par...
3,https://nordpresse.be/jean-paul-delevoye-sexpl...
4,https://nordpresse.be/jean-paul-delevoye-nomme...


In [120]:
df_nordpresse_pages.shape

(245, 1)

#### Scraping news

In [121]:
# clean last output
clean_file(PATH_NEWS_NORDPRESSE)

File ../../data/nordpresse.json does not exist!


In [122]:
# configure : add all retrieved links to Spider
NordpresseSpider.start_urls = df_nordpresse_pages["link"].tolist()
NordpresseSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_NORDPRESSE
    }
# scraping
run_spider(NordpresseSpider)

In [123]:
import pandas as pd

df_nordpresse = pd.read_json(PATH_NEWS_NORDPRESSE)
df_nordpresse.head()

,author,body,date_published,source,theme,title,url
0,Frodolphe Amplebedaine,"Paris, centre de la France, concentre trop d’i...",2019-12-03,Nordpresse,Start Up Nation,Déconcentration : l’ENA à Toulouse !,https://nordpresse.be/deconcentration-lena-a-t...
1,ToLa,Le Haut-Commissaire aux retraites Jean-Paul De...,2019-12-21,Nordpresse,France,J.-P. Delevoye toujours au gouvernement: « J’a...,https://nordpresse.be/j-p-delevoye-toujours-au...
2,Vincent Flibustier,C’est un retournement de situation incroyable ...,2019-11-28,Nordpresse,France,Rerebondissement: Aurore Bergé veut aussi deve...,https://nordpresse.be/rerebondissement-aurore-...
3,Stephane VERON,C’est une victoire inattendue du Gouvernement...,2019-12-18,Nordpresse,Politique,Retraites – Découragés par le nom de leur nouv...,https://nordpresse.be/retraites-decourages-par...
4,Stephane VERON,C’est une séance privée dans la salle de ciné...,2019-11-18,Nordpresse,France,"Ému par « La Soif de l’Or », Macron annonce un...",https://nordpresse.be/emu-par-la-soif-de-lor-m...


In [124]:
df_nordpresse.shape

(245, 7)

In [187]:
df_nordpresse[df_nordpresse["body"].isna()].shape[0]

32

## Scraping buzzbeed.com

### Class definitions

In [172]:
import scrapy
from scrapy.loader import ItemLoader
from scrapy.loader.processors import MapCompose, Join, TakeFirst
from w3lib.html import remove_tags


class BuzzbeedItem(scrapy.Item):
    '''
    Class item to declare different information to scrap
    and how to process (as input or output) for buzzbeed.com
    '''
    # define the fields for your item here like:
    url = scrapy.Field(output_processor=TakeFirst())
    
    source = scrapy.Field(output_processor=TakeFirst())
    
    author = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    title = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    theme = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=lambda v: v[-1]
    )
    
    date_published = scrapy.Field(
        input_processor=MapCompose(remove_tags),
        output_processor=TakeFirst()
    )
    
    description = scrapy.Field(output_processor=TakeFirst())
    
    body = scrapy.Field(
        input_processor=MapCompose(remove_tags, 
                                   lambda v: v.replace(u'\xa0', u' '),
                                  lambda v: v.strip()),
        output_processor=Join()
    )

In [173]:
class BuzzbeedSpider(scrapy.Spider):
    '''
    Spider to scrap over buzzbeed webpages : 
    - how to find information for scraping
    - which field names to store
    '''
    # Your spider definition
    name = 'news_buzzbeed_spider'
    # output definition :
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'items_buzzbeed.json'
    }
    
    # urls to scrap
    start_urls = [
        'https://www.buzzbeed.com/'
    ]

    def parse(self, response):
        '''
        Parse definition with xpath which define all patterns to use
        for retrieve information into HTML strings
        '''
        #url	source	author	title	theme	description	date_published	body   
        l = ItemLoader(item=BuzzbeedItem(), selector=response)
        
        l.add_value('url', response.url)
        
        l.add_value('source', "Buzzbeed")
        
        #<span class="author vcard">
        # <a class="url fn n" href="https://www.buzzbeed.com/author/scoub">Jean Deumerez</a>
        #</span>
        l.add_xpath('author', 
            '//span[@class="author vcard"]/a')
        
        l.add_xpath("title", "//h1")
        
        #<span class="cat-links">
        # <a href="https://www.buzzbeed.com/category/sentiment-dinsecurite" rel="category tag">Sentiment d'insécurité</a>
        #</span>
        l.add_xpath("theme", '//span[@class="cat-links"]/a')
        
        #l.add_xpath("description", ".//div[@class='intro']/p")
        l.add_value('description', "")
        
        #<time class="entry-date published" datetime="2019-03-27T16:01:03+01:00">mars 27, 2019</time>
        l.add_xpath("date_published", '//time/@datetime',
                    re="[0-9]+-[0-9]+-[0-9]+")
        #'//div[@class="entry-content"]/div'
        l.add_xpath("body",
            '//div[@class="entry-content"]/div|//div[@class="entry-content"]/p')
        
        
        yield l.load_item()

In [174]:
import scrapy

class BuzzbeedRssSpider(scrapy.Spider):
    '''
    Class Spider for retrieving all links to news webpages from buzzbeed RSS
    '''
    name = "buzzbeed_rss"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'buzzbeed_rss.json'
    }
    
    def start_requests(self):
        urls = [
                'https://www.buzzbeed.com/feed',
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
     
    def parse(self, response):
        for post in response.xpath('//channel/item'):
            yield {
                'title' : post.xpath('title//text()').extract_first(),
                'link': post.xpath('link//text()').extract_first(),
                'pubDate' : post.xpath('pubDate//text()').extract_first(),
            }


In [175]:
import scrapy
import re

class BuzzbeedPageSpider(scrapy.Spider):
    '''
    Spider to scrap all Buzzbeed pages from selected category
    Configure : 
    - BuzzbeedPageSpider.custom_settings : save location 
    - num_max_pages : the number of next page to scrap
    - url_first_page : web page to start with
    '''
    name = "buzzbeed_page"
    
    custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': 'pages_buzzbeed.json'
    }
    
    num_max_pages = 1
    
    url_first_page = 'https://www.buzzbeed.com/'
    
    def start_requests(self):
        urls = [
                self.url_first_page,
        ]
        for url in urls:
            yield scrapy.Request(url=url, callback=self.parse)
    
    def parse(self, response):
        
        # //h2[@class="entry-title"]/a/@href
        for post in response.xpath('//h2[@class="entry-title"]'):
            
            #print('post : ', post) # TEST
            yield {
                'link': post.xpath('a/@href').extract_first()
            }
        # //a[@class="next page-numbers"]/@href
        next_page = response.xpath(
            '//a[@class="next page-numbers"]/@href').get()
        #print('next_page : ', next_page) # TEST
        if next_page is not None:
            try:
                # ex: https://www.buzzbeed.com/page/3
                num_next_page = int(re.search("(?<=/)\d+$", 
                         next_page).group(0))
                
                if (num_next_page < self.num_max_pages):
                    #next_page = response.urljoin(next_page)
                    yield scrapy.Request(next_page, callback=self.parse)
            except:
                next_page = None
                

### Scraping RSS urls

#### Definitions

In [170]:
# path to results file for urls
PATH_RSS_PAGES_BUZZBEED = '../../data/buzzbeed_rss_urls.json'
# declare your data location for scraping rss links 
PATH_RSS_NEWS_BUZZBEED = '../../data/buzzbeed_rss_pages.json'

#### Scraping RSS urls links

In [171]:
# clean (move file if exist)
clean_file(PATH_RSS_PAGES_BUZZBEED)

File ../../data/buzzbeed_rss_urls_20191227_18_15_52.json moved!


In [134]:
# Configure spider
BuzzbeedRssSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_PAGES_BUZZBEED
}
# scraping
run_spider(BuzzbeedRssSpider)

In [135]:
import pandas as pd

df_rss_buzzbeed = pd.read_json(PATH_RSS_PAGES_BUZZBEED)
df_rss_buzzbeed

,link,pubDate,title
0,https://www.buzzbeed.com/marine-le-pen-il-ny-a...,"Wed, 27 Mar 2019 15:01:03 +0000",Marine Le Pen : “il n’y a pas de problème d’im...
1,https://www.buzzbeed.com/emmanuel-macron-surpr...,"Wed, 20 Feb 2019 19:59:23 +0000",Emmanuel Macron surpris en train de dessiner d...
2,https://www.buzzbeed.com/lauteur-presume-des-a...,"Wed, 12 Dec 2018 21:35:24 +0000",L’auteur présumé des attentats de Strasbourg s...
3,https://www.buzzbeed.com/bernard-werber-les-au...,"Sat, 17 Nov 2018 21:37:10 +0000",Bernard Werber : « Les auteurs de Science-Fict...
4,https://www.buzzbeed.com/emmanuel-macron-rend-...,"Mon, 12 Nov 2018 20:58:20 +0000",Emmanuel Macron rend hommage aux 6 millions d’...
5,https://www.buzzbeed.com/un-neo-nazi-deguise-e...,"Fri, 28 Sep 2018 17:25:48 +0000",Un néo-nazi déguisé en sauveteur de Juifs à Ch...
6,https://www.buzzbeed.com/la-france-islamophobe...,"Tue, 21 Aug 2018 17:58:11 +0000",La France islamophobe : 75% des détenus seraie...
7,https://www.buzzbeed.com/le-qi-en-baisse-a-cau...,"Tue, 21 Aug 2018 17:01:22 +0000",Le QI en baisse à cause de l’Américanisation
8,https://www.buzzbeed.com/yemen-la-bombe-qui-a-...,"Sun, 19 Aug 2018 13:24:28 +0000",Yémen : La bombe qui a tué 40 futurs poètes et...
9,https://www.buzzbeed.com/les-syriens-peuvent-a...,"Sun, 15 Jul 2018 15:59:12 +0000",Les Syriens peuvent à nouveau vivre dans leur ...


In [136]:
df_rss_buzzbeed.shape

(10, 3)

#### Scraping RSS news

In [157]:
# clean (move file if exist)
clean_file(PATH_RSS_NEWS_BUZZBEED)

File ../../data/buzzbeed_rss_pages_20191227_18_03_47.json moved!


In [158]:
# configure : add all retrieved links to Spider
BuzzbeedSpider.start_urls = df_rss_buzzbeed["link"].tolist()
BuzzbeedSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_RSS_NEWS_BUZZBEED
}

# scraping
run_spider(BuzzbeedSpider)

In [159]:
import pandas as pd

df_buzzbeed = pd.read_json(PATH_RSS_NEWS_BUZZBEED)
df_buzzbeed.head()

,author,body,date_published,source,theme,title,url
0,Jean Deumerez,Marine Le Pen continue de surprendre son élec...,2019-03-27,Buzzbeed,Sentiment d'insécurité,Marine Le Pen : “il n’y a pas de problème d’im...,https://www.buzzbeed.com/marine-le-pen-il-ny-a...
1,Francis Lépante,"« La proximité du marché de Noël, et surtout ...",2018-12-12,Buzzbeed,Uncategorized,L’auteur présumé des attentats de Strasbourg s...,https://www.buzzbeed.com/lauteur-presume-des-a...
2,Francis Lépante,L’auteur du livre “Les fourmis” a un avis tra...,2018-11-17,Buzzbeed,Uncategorized,Bernard Werber : « Les auteurs de Science-Fict...,https://www.buzzbeed.com/bernard-werber-les-au...
3,Francis Lépante,"Hier, Emmanuel Macron a rendu hommage aux 6 m...",2018-11-12,Buzzbeed,Uncategorized,Emmanuel Macron rend hommage aux 6 millions d’...,https://www.buzzbeed.com/emmanuel-macron-rend-...
4,Jean Deumerez,Une nouvelle preuve que notre pays est bien r...,2018-08-21,Buzzbeed,Uncategorized,La France islamophobe : 75% des détenus seraie...,https://www.buzzbeed.com/la-france-islamophobe...


In [160]:
df_buzzbeed.shape

(10, 7)

In [161]:
df_buzzbeed.at[0,"url"]

'https://www.buzzbeed.com/marine-le-pen-il-ny-a-pas-de-probleme-dimmigration-en-france'

In [163]:
df_buzzbeed.at[0,"body"]

' Marine Le Pen continue de surprendre son électorat, affirmant, apès avoir nié être au courant du phénomène de Grand-Remplacement tel qu’exposé par René Camus, que elle n’a “jamais été au courant de problèmes d’immigration en France”. Elle achève ainsi haut la main sa campagne de dédiabolisation.  “En France il y a peut-être tout au plus quelques descendants d’immigrés, mais mon attention n’a jamais été attiré par une présence significative d’immigrés en France. En outre, ce sont des immigrés qui travaillent, contribuent à la culture du pays, ne touchent pas plus d’allocations que le Francais moyen, bref, on verrait mal au rassemblement national en quoi l’immigration serait un problème pour la France. C’est plutôt une chance, une opportunité.”  Marine Le Pen a pourtant dérapé sur l’Allemagne: “les Allemands, ce sont les heures sombres de notre histoire, ils veulent toujours nous reprendre l’Alsace-Lorraine, et là, avec l’euro, c’est le même projet contre les nations qui recommence, c’

### Scraping urls into webpages 

#### Definitions

In [176]:
URL_PAGES_BUZZBEED = 'https://www.buzzbeed.com/'
# declare your data location
PATH_PAGES_BUZZBEED = '../../data/pages_buzzbeed.json'
PATH_NEWS_BUZZBEED = '../../data/buzzbeed.json'

#### Scraping url links

In [177]:
# clean (move file if exist)
clean_file(PATH_PAGES_BUZZBEED)

File ../../data/pages_buzzbeed.json does not exist!


In [178]:
# configure scraping
BuzzbeedPageSpider.url_first_page = URL_PAGES_BUZZBEED
BuzzbeedPageSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_PAGES_BUZZBEED
    }
BuzzbeedPageSpider.num_max_pages = 25 # 10 articles by page

# clean last output
clean_file(PATH_PAGES_BUZZBEED)
# scraping page urls
run_spider(BuzzbeedPageSpider)

File ../../data/pages_buzzbeed.json does not exist!


In [179]:
import pandas as pd

df_buzzbeed_pages = pd.read_json(PATH_PAGES_BUZZBEED)
df_buzzbeed_pages.head()

,link
0,https://www.buzzbeed.com/marine-le-pen-il-ny-a...
1,https://www.buzzbeed.com/emmanuel-macron-surpr...
2,https://www.buzzbeed.com/lauteur-presume-des-a...
3,https://www.buzzbeed.com/bernard-werber-les-au...
4,https://www.buzzbeed.com/emmanuel-macron-rend-...


In [180]:
df_buzzbeed_pages.shape

(240, 1)

#### Scraping news

In [181]:
# clean last output
clean_file(PATH_NEWS_BUZZBEED)

File ../../data/buzzbeed.json does not exist!


In [182]:
# configure : add all retrieved links to Spider
BuzzbeedSpider.start_urls = df_buzzbeed_pages["link"].tolist()
BuzzbeedSpider.custom_settings = {
      'FEED_FORMAT': 'json',
      'FEED_URI': PATH_NEWS_BUZZBEED
    }
# scraping
run_spider(BuzzbeedSpider)

In [183]:
import pandas as pd

df_buzzbeed = pd.read_json(PATH_NEWS_BUZZBEED)
df_buzzbeed.head()

,author,body,date_published,source,theme,title,url
0,Jean Deumerez,"Le Président Francais, Emmanuel Macron, a été...",2019-02-20,Buzzbeed,Insolite,Emmanuel Macron surpris en train de dessiner d...,https://www.buzzbeed.com/emmanuel-macron-surpr...
1,Francis Lépante,L’auteur du livre “Les fourmis” a un avis tra...,2018-11-17,Buzzbeed,Uncategorized,Bernard Werber : « Les auteurs de Science-Fict...,https://www.buzzbeed.com/bernard-werber-les-au...
2,Francis Lépante,"« La proximité du marché de Noël, et surtout ...",2018-12-12,Buzzbeed,Uncategorized,L’auteur présumé des attentats de Strasbourg s...,https://www.buzzbeed.com/lauteur-presume-des-a...
3,Francis Lépante,"Hier, Emmanuel Macron a rendu hommage aux 6 m...",2018-11-12,Buzzbeed,Uncategorized,Emmanuel Macron rend hommage aux 6 millions d’...,https://www.buzzbeed.com/emmanuel-macron-rend-...
4,Jean Deumerez,Preuve qu’ils sont partout. « Ils se cachent ...,2018-09-28,Buzzbeed,Uncategorized,Un néo-nazi déguisé en sauveteur de Juifs à Ch...,https://www.buzzbeed.com/un-neo-nazi-deguise-e...


In [184]:
df_buzzbeed.shape

(240, 7)

In [186]:
df_buzzbeed[df_buzzbeed["body"].isna()].shape[0]

0

## Export Results

### Clean results

In [198]:
import pandas as pd

df_gorafi_soc = pd.read_json(PATH_NEWS_GORAFI_SOCIETE)
df_gorafi_pol = pd.read_json(PATH_NEWS_GORAFI_POLITIQUE)
df_nordpresse = pd.read_json(PATH_NEWS_NORDPRESSE)
df_buzzbeed = pd.read_json(PATH_NEWS_BUZZBEED)

In [199]:
df_fake = pd.concat([df_gorafi_soc, df_gorafi_pol, df_nordpresse, 
         df_buzzbeed], ignore_index=True)
df_fake

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,body,date_published,description,source,theme,title,url
0,La Rédaction,« Mes équipes sentaient le malaise s’installer...,19/12/2019,"Afin d’être plus en phase avec l’actualité, le...",LeGorafi,Société,"Noël – Pour coller à la réalité, les magasins ...",http://www.legorafi.fr/2019/12/19/noel-pour-co...
1,La Rédaction,"Oui, car Mathias a économisé pas moins de 5 eu...",10/12/2019,"Alors qu’il rentrait d’un week-end, Mathias a ...",LeGorafi,Société,Il économise 5 euros en achetant au duty-free ...,http://www.legorafi.fr/2019/12/10/il-economise...
2,La Rédaction,1/ Faire un beau chèque en s’appliquant pour l...,17/12/2019,L’échec amoureux n’a plus de secret pour vous ...,LeGorafi,Société,Psycho : Comment guérir d’un chagrin d’amour e...,http://www.legorafi.fr/2019/12/17/psycho-comme...
3,La Rédaction,\nwindow.beOpAsyncInit = function() {\n BeOpS...,12/12/2019,"Depuis les annonces d’Edouard Philippe, vous v...",LeGorafi,Société,Test : quel est l’âge idéal pour mourir avant ...,http://www.legorafi.fr/2019/12/12/test-quel-es...
4,La Rédaction,"Selon les services sanitaires, se sont plusieu...",11/12/2019,Paris – La chaîne de restauration rapide lancé...,LeGorafi,Société,Plusieurs Burger Quiz fermés après une visite ...,http://www.legorafi.fr/2019/12/11/plusieurs-bu...
5,La Rédaction,"Il y aura eu des joies, des peines, de l’espoi...",04/12/2019,"Après des années de recherche, Richard va enfi...",LeGorafi,Société,Depuis 6 ans il tourne dans un parking Vinci p...,http://www.legorafi.fr/2019/12/04/depuis-6-ans...
6,La Rédaction,Ainsi les chauffeurs sont autorisés à prendre ...,19/12/2019,Paris – Pour aider les Français à pouvoir prof...,LeGorafi,Société,Le gouvernement autorise les chauffeurs des bu...,http://www.legorafi.fr/2019/12/19/le-gouvernem...
7,La Rédaction,C’est lors du cours d’anglais de M. Pochat que...,02/12/2019,La rumeur s’est répandue tôt ce matin comme un...,LeGorafi,Société,Collège – Le nouveau a l’air vraiment bizarre,http://www.legorafi.fr/2019/12/02/college-le-n...
8,La Rédaction,« L’idée n’est évidemment pas de restreindre l...,14/11/2019,Après l’annulation de la venue de Sylviane Aga...,LeGorafi,Société,Universités : les conférenciers invités pour d...,http://www.legorafi.fr/2019/11/14/universites-...
9,La Rédaction,Agissant avec une cruauté sans pareille et un ...,29/11/2019,(El Mundo Today) – Des milliers de pétroliers ...,LeGorafi,Société,Greta Thunberg coule cinq pétroliers et détour...,http://www.legorafi.fr/2019/11/29/greta-thunbe...


#### Delete NaN body

In [204]:
df_fake[df_fake["body"].isna()].shape[0]

32

In [205]:
df_fake.dropna(subset=['body'], inplace=True)

In [206]:
df_fake[df_fake["body"].isna()].shape[0]

0

In [208]:
df_fake.shape[0]

945

In [211]:
df_fake["body"].nunique()

928

In [213]:
df_fake["body"].value_counts()

\nwindow.beOpinionAsyncInit = function() {\n  BeOpinionSDK.init({\n    account: "589c71a4d068331ee05ac9fd"\n  });\n  BeOpinionSDK.watch();\n};\n\n                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [214]:
str_test = '\nwindow.beOpinionAsyncInit = function() {\n  BeOpinionSDK.init({\n    account: "589c71a4d068331ee05ac9fd"\n  });\n  BeOpinionSDK.watch();\n};\n\n' 

In [215]:
df_fake[df_fake["body"] == str_test].shape[0]

13

In [216]:
df_fake[df_fake["body"] == str_test]["title"]

13      Test : par quel robot allez-vous être remplacé ?
44                  Test – Êtes-vous une Fiat Multipla ?
49                     Test : Mais où est donc Ornicar ?
54     Test – Vous n’en avez pas marre de répondre à ...
60     Test : Dans quelles atroces souffrances allez-...
63      Test : Quel tracteur Massey Ferguson êtes-vous ?
80     Test – Êtes vous capable de survivre au Servic...
112                              Test : Qui a le droit ?
120    TEST : Etes-vous un rebelle à moto ou un compt...
170    Test : êtes-vous un influenceur ou parlez-vous...
208         Test : Êtes-vous une pourriture communiste ?
258          Test : quel ministre vous méprise le plus ?
332    Test : Quel ministère Emmanuel Macron va-t-il ...
Name: title, dtype: object

In [218]:
df_fake[df_fake["body"].str.match("^Test")]["title"]

Series([], Name: title, dtype: object)

In [226]:
df_fake[df_fake["title"].str.startswith('Test')].shape[0]

16

In [224]:
df_fake[df_fake["body"] == str_test]['title'].tolist()

['Test : par quel robot allez-vous être remplacé ?',
 'Test – Êtes-vous une Fiat Multipla ?',
 'Test : Mais où est donc Ornicar ?',
 'Test – Vous n’en avez pas marre de répondre à des tests sans intérêt au lieu de réfléchir un peu à vos projets de vie ?',
 'Test : Dans quelles atroces souffrances allez-vous mourir ?',
 'Test : Quel tracteur Massey Ferguson êtes-vous ?',
 'Test – Êtes vous capable de survivre au Service National Universel ?',
 'Test : Qui a le droit ?',
 'TEST : Etes-vous un rebelle à moto ou un comptable ?',
 'Test : êtes-vous un influenceur ou parlez-vous correctement français ?',
 'Test : Êtes-vous une pourriture communiste ?',
 'Test : quel ministre vous méprise le plus ?',
 'Test : Quel ministère Emmanuel Macron va-t-il vous proposer ce week-end ?']

In [228]:
df_fake[df_fake["title"].str.startswith('Test')].url.tolist()

['http://www.legorafi.fr/2019/12/12/test-quel-est-lage-ideal-pour-mourir-avant-la-retraite/',
 'http://www.legorafi.fr/2019/11/04/test-par-quel-robot-allez-vous-etre-remplace/',
 'http://www.legorafi.fr/2019/09/10/test-etes-vous-une-fiat-multipla/',
 'http://www.legorafi.fr/2019/09/03/test-mais-ou-est-donc-ornicar/',
 'http://www.legorafi.fr/2019/08/26/test-vous-nen-avez-pas-marre-de-repondre-a-des-tests-sans-interet-au-lieu-de-reflechir-un-peu-a-vos-projets-de-vie/',
 'http://www.legorafi.fr/2019/07/10/test-dans-quelles-atroces-souffrances-allez-vous-mourir/',
 'http://www.legorafi.fr/2019/07/23/quel-tracteur-massey-ferguson-etes-vous/',
 'http://www.legorafi.fr/2019/06/20/test-etes-vous-capable-de-survivre-au-service-national-universel/',
 'http://www.legorafi.fr/2019/04/10/test-qui-a-le-droit/',
 'http://www.legorafi.fr/2019/02/20/test-etes-vous-devant-un-ecran/',
 'http://www.legorafi.fr/2019/02/06/test-etes-vous-sylvain-naudac-de-saint-brieuc/',
 'http://www.legorafi.fr/2019/01/28